# An example of using etasl_simulator

This notebook provides the same as the etasl-example_1.ipynb notebook, but only keeps the functional code and leaves out all explanation. Can be used as a template for your own experiments.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Simulating the eTaSL controller using the high-level eTaSL driver

Import all the necessary classes and functions from the etasl_py.etasl python module:

In [ ]:
from etasl_py.etasl import etasl_simulator,array_to_dict,dict_to_array,to_deg, to_rad
e = etasl_simulator(regularization_factor= 0.00001)
e.readTaskSpecificationFile("etasl_robot_ur10.lua")
e.readTaskSpecificationFile("etasl_simple_task.lua")
pos_lbl = ['shoulder_pan_joint','shoulder_lift_joint', 'elbow_joint', 
           'wrist_1_joint', 'wrist_2_joint',  'wrist_3_joint', 'f1']
initial_jpos = np.array([0, -np.pi*0.6 , np.pi*0.6,0,0,0,0])
N=4000
dt=0.005
time = np.arange(0,N)*dt
inp_lbl=['tgt_x','tgt_y','tgt_z']
inp=np.zeros((N, len(inp_lbl)))
inp[:,0] = np.sin(time)*0.15 + 0.7
inp[:,1] = time*0
inp[:,2] = time*0 + 0.3
inpvel = np.zeros((N, len(inp_lbl)))
inpvel[:,0] = np.cos(time)*0.15
inpvel[:,1] = time*0
inpvel[:,2] = time*0
e.setInputTable(inp_lbl,inp,inpvel)
e.initialize(initial_jpos, pos_lbl)
e.simulate(N=N,dt=dt)

# Plotting (using the Bokeh library to provide interactive plots)

In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

def plotv(TIME,VAR, VAR_LBL, plotw=450,ploth=250, ncols=2, tooltip="$x : $y"):
    p=[]
    for i in range(0,VAR.shape[1]):
        f = figure(x_axis_label='time [s]', 
                y_axis_label= VAR_LBL[i],     
                tooltips=tooltip)
        if i==0:
            fx=f.x_range
        else:
            f.x_range=fx
        f.line(TIME, VAR[:,i], line_width=2)
        p.append(f)

    pg = gridplot( p, ncols=ncols , plot_width=plotw, plot_height=ploth) 
    show(pg)

plotv(e.TIME,e.POS,e.POS_LBL)
plotv(e.TIME,e.VEL,e.POS_LBL)

plotw=450
ploth=250
ncols=2

p1 = figure(x_axis_label='time', 
            y_axis_label= "error x", tooltips="$x : $y")
p1.line(e.TIME, e.OUTP[:,0] - e.INP[:,0], line_width=2)

p2 = figure(x_axis_label='time', 
            y_axis_label= "error y", tooltips="$x : $y",
            x_range = p1.x_range)
p2.line(e.TIME, e.OUTP[:,1] - e.INP[:,1], line_width=2)

p3 = figure(x_axis_label='time', 
            y_axis_label= "error z", tooltips="$x : $y",
            x_range = p1.x_range)
p3.line(e.TIME, e.OUTP[:,2] - e.INP[:,2], line_width=2)

pg = gridplot(  [p1,p2, p3], plot_width=plotw, plot_height=ploth, ncols=ncols)    
show(pg)